# Imaging biomarker extraction walkthrough
This notebook will go over some examples of extacting biomarkers from structural T1 weighted MR images and amyloid PET data. These are some of the most widely used biomarkers that are used in AD research. They are also measures used in the EBM. There are many different publicly available, open-source software packges that you can use to make these measures from T1w images.  The specific techniques that we are using in this notebook are meant to demonstrate the key concepts involved in the processing pipelines, but they have not been optimised for this task. The exact decision of what software and pipeline to use for your research  will depend on what works best for your data and application.

## Preparation
First, we will import the packages that will be used in our notebook. We will be using the following packages:
* [nibabel](https://nipy.org/nibabel/) - a handy package to work with neuroimaging data.
* [Advanced Normalisation Toolbox (ANTs)](https://antspy.readthedocs.io/en/latest/index.html) - a popular set of tools that do registration, segmentation and other image processing techniques.   

In [ ]:
%%capture --no-display
import os
import nibabel as nb
import numpy as np
from niwidgets import NiftiWidget
import pandas
import ants

## Imaging basics
The first step that we are going to do is load in an image and have a quick look at it. There are many different file formats used in medical imaging data, but the two most common are the _Digital Image and COmmunication in Medicine_ ([DICOM](https://www.dicomstandard.org/current))and the _Neuroimaging InFormatics Technology Initiative_ ([NIFTI](https://nifti.nimh.nih.gov/#:~:text=NIfTI%2D1%20is%20a%20new,MRI%20data%20analysis%20software%20packages.). In both case, data is stored as a flat sheet (two-dimensional image data) or a cube (three-dimensional) where individual elements of the data are called pixels (2-D) or voxels (3-D) where one or more numerical values are associated with each element. These values could represent the intensity, a time series of intensities, color values (RGB), vectors, or other different measurements. Associated with this data is _metadata_ which describes key aspects of the data. This metadata could describe, among other things, how the data was acquired, information about the patient, how big the image is, where in physical space of the scanner the image is located, and how data in each pixel is represented.


### Opening an image
Let's first load in an image stored in the Nifti file format into memory and see some basic information about it. One nice thing about Nifti is that you can _compress_ the image using gzip, which makes the image much smaller on your hard drive (roughoy 30-50% for images and up to 90-95% for binary masks), but most software packages can still read it without decomressing it (it will still take up the same amount of memory on your computer when you load it in as part of a program though)

**Note: If you want to run this yourself while on the course, download the data from the TeamCoders Team channel and change the data_root variable to match the location on your computer**


In [ ]:
data_root = "/Users/davecash/Data/IDEAS/TeamCoder_EBM/bids"
subject_id =  "011-S-4906"
bids_desc = "t1"
subject_root = os.path.join(data_root,"sub-"+subject_id)
t1_img_name = os.path.join(subject_root,"anat","sub-" + subject_id + "_desc-" + bids_desc + ".nii.gz")
t1_img = nb.load(t1_img_name)
print(t1_img.shape)


This line of output comes from printing out the field shape from the Nifti image file. It says we have a cube of imaging data in the file that contains 208 rows, 256 columns, and 256 slices.  Now lets find out what this image data contains. I'm going to find a test voxel and ask for its intensity. We typically denote voxel locations as *i*, *j*, and *k*. I have chosen voxel location (100,100,100) which means I want to know the intensity of the voxel at row 100 and column 100 within slice 100. 


In [ ]:
test_voxel = (100,100,100)
t1_img_data = t1_img.get_fdata()
print(t1_img_data[test_voxel])

Python starts counting at **zero**, so this row 100 is actually the 101st row in the matrix. It also means that the indexes used to access the voxels inside the array go frrom rows 0 to 207, not 1 to 208, and columns from 0 to 255, not 1 to 256. Watch what happens if I want to get the first row and column in the last slice if I think Python is counting from 1.

In [ ]:
first_voxel_in_last_slice=(1,1,256)
print(t1_img_data[first_voxel_in_last_slice])

So what range of intensities are there in this image volume? We can get that by asking what the minimum and maximum intensities are.

In [ ]:
print(t1_img_data.min())
print(t1_img_data.max())

Now we are able to look at voxels, but how do we know where they are in the real world? An MRI of a fruit fly or an elephant could contain 256 slices, but one image would have to represent a much larger size in the real world than the other. What if the elephant was sideways in the scanner, but the fruitfly had its head towards the top of the scanner and its tail at the bottom? Then the rows and columns might mean entirely different anatomical directions!

Embedded in the images metadata are not only the dimensions of each voxel, but also an important transformation, or mapping, that tells us how to convert the voxel location *(i,j,k)* to the real-world coordinates of *(x,y,z)*. Understanding this mapping is very important, as we will be needing this information to align images later. This transformation is stored in the affine field. 

The real-world coordinate system tends to be defined according to the patient. The x-axis tends to go from patient left to patient right, the y axis tends to go from anterior to posterior, and the z-axis goes from top to bottom of the patient. 

In [ ]:
np.set_printoptions(suppress=True)
print(t1_img.affine)

What we see here is a four by four matrix. The first 3 rows and columns represent how we *rotate* and *scale* the image voxel to get it into real worlsd space. Here all the numbers off the main diagonal are 0, indicating that there is no rotation, so the image is in an orientation that is aligned with the cardinal axes of the scanner in the real world. That means that the rows of voxels go from left to right in the real world and the columns go from the patients front to the patients back, and finally the slices go from the top of the head down towards the neck. When there is no rotation, than it is really easy to understand how big each indivudal voxel, as the values in the diagonal will indicate how big the voxel is. These represent effectively that each voxel is 1.2 x 1.0 x 1.0mm in size. So when we move one voxel over in the X direction, we are moving 1.2mm in real world space, and in the other directions we are moving 1.0 mm in real world space. . 

The first three numbers in the final column that are all quite large represent the *translation* or shift we need to move the voxel into real-world space. The bottom row is always 0,0,0,1 and is used to make it easier to map one or thousands of points in one single matrix operation. 

How do we get the equivalent real-world coordinates from the voxel location? Through a process of matrix multiplication

In [ ]:
real_world = t1_img.affine.dot(list(test_voxel) + [1])
print(real_world)

So this means that the voxel (100,100,100) represents physical space (16.4 mm. -2.7 mm, -52.8 mm). When we have different kinds of images acquired in the scanning session or in other scanning sessions, then may have different orientations or different voxel sizes, then it makes it easier to compare and overlay the images together. We will see a bit more abotu that when we discuss registration. Now that we know a bit more about how to traverse the images and what they mean. Let's take a look at one! The niwdiget below plots it in your notebook. However, you can also look at it in the command line using the tool fsleyes

In [ ]:
niwidget = NiftiWidget(t1_img_name)
niwidget.nifti_plotter()

## Bias correction
Notice how in the white matter you can see some variation in intensity. It slowly varies from darker areas to lighter areas. This is due to the fact that the static magnetic field, known as $B_0$, is not perfectly constant everywhere. There are small fluctuations that slowly vary over the volume of the head that cause these subtle differences in intensity. These  differences can cause issues for subsequent processing routines needed to get accurate measures of brain volume. 

In order to reduce this effect, we employ a technique known as d _bias correction_, which  normalises the intensity across the image. For this tutorital, we are going to be using one of the most common bias correction algorithms out there, [N3](https://www.nitrc.org/docman/view.php/6/880/sled.pdf). Other software packages, like SPM, integrate bias correction with the[tissue segmetnation](#tissue-segmentation) step.  

The code below will perform the bias correction and plot out images of the original image int he top row, the corrected image in the middle row and the difference between them in the bottom row. Notice how smooth the bias field is across the image. 

Let's also save the T1 N3 and T1_diff outputs so that we can see them in fsleyes. 


In [ ]:

t1_img = ants.image_read(t1_img_name)
t1_n3 = ants.n3_bias_field_correction(t1_img)
t1_diff = t1_img - t1_n3
ants.plot_ortho_stack([t1_img,t1_n3,t1_diff])
bids_desc = "t1-n3"
t1_n3_name = os.path.join(subject_root,"anat","sub-" + subject_id + "_desc-" + bids_desc + ".nii.gz")
ants.image_write(t1_n3,t1_n3_name)
bids_desc = "bias-field"
t1_diff_name = os.path.join(subject_root,"anat","sub-" + subject_id + "_desc-" + bids_desc + ".nii.gz")
ants.image_write(t1_diff,t1_diff_name)


## Brain extraction
Before we perform the [tissue segmentation](#tissue-segmentation) step, we are going to do a bit mroe pre-processing. Some tissue segmetnation algorithms require the removal of most of the non-brain structures (neck, eyes, skull, dura, eta) leacing only the brain parenchyma and surrounding cerebrospinal fluid.

There are many different ways to do this, bt we are going to a very simple method to illustrate the process. More advanced methods, which might work better for your analysis that you want to do, are available.

We are going to grap a standard atlas called MNI152. This comes with a handy mask. We are going to register this to our image and then use a predefined mask as our starting point.
But look at the images below (and in fsleyes). You can see one is much bigger than the other and they are not in the same orientation.

But before we do this and other steps, it's a good idea to do a step called _skull stripping_, where we remove all the bits from outside the brain. Again thare are lots of different ways to do this, but we are going to take a very simple approach for the time being. The following should show the original, the corrected, and then the difference image so that you can see the bias field. Also open these images up in dsleyes so that you can toggle back and forth and see the images. 

In [ ]:
atlasdir = os.path.join(data_root,"atlas")
mni_img_file = os.path.join(atlasdir,"MNI152_T1_1mm.nii.gz")
mni_img = ants.image_read(mni_img_file)
mni_mask_file = os.path.join(atlasdir,"MNI152_T1_1mm_brain_mask_dil.nii.gz")
mni_mask = ants.image_read(mni_mask_file)
ants.plot_ortho_stack([mni_img,t1_n3])


So now we are going to run a registration to get information from the standard atlas into the space of our image. We will discuss [registerion](#co-registration) later. Registration can be a tricky thing to do if we don't give it a good starting point, so we are going to first run an affine initialiser which will find a good starting point for us. 

In [ ]:
init_tx_file = ants.affine_initializer(fixed_image = mni_img, moving_image = t1_n3, mask = mni_mask)
init_tx = ants.read_transform(init_tx_file)
t1_mni_init = ants.apply_transforms(fixed = mni_img, moving = t1_n3,transformlist=init_tx_file)


Now let's take a look at the image and see if we have improved.

In [ ]:
ants.plot_ortho_stack([mni_img,t1_mni_init])

That seems to have them more aligned! Now let's do the full registration just to see if we can improve it further.

In [ ]:
affine = ants.registration(fixed = mni_img, moving = t1_n3, type_of_transform = "Affine", initial_transform = init_tx_file, mask = mni_mask)

Now what does that registration produce? Let's print out the return value to see what it holds

In [ ]:
print(affine)

Affine holds four key outputs (from the ANTS documentation):
* _warpedmovout_: Moving image warped to space of fixed image. 
* _warpedfixout_: Fixed image warped to space of moving image. 
* _fwdtransforms_: Transforms to move from moving to fixed image. 
* _invtransforms_: Transforms to move from fixed to moving image.

To see what the registration has done let us look at the mni image and the moxing image (our original image) transformed to MNI.

In [ ]:
ants.plot_ortho_stack([mni_img,affine['warpedmovout']])

That looks really nice. Ants provides us both the _forward mapping_, moving our image into the coordinate system described by MNI and the _inverse mapping_, moving the MNI information back into our image. This allows use to take a mask from MNI and put it into our space.

In [ ]:
t1_mask = ants.apply_transforms(fixed = t1_n3, moving = mni_mask,transformlist=affine['fwdtransforms'],interpolator="nearestNeighbor",whichtoinvert=[True])
ants.plot_ortho(t1_n3,overlay=t1_mask)

That looks pretty decent. We aren't missing any brain and so we can proceed with removing everything outside of this mask.

In [ ]:
t1_brain = ants.mask_image(t1_n3,t1_mask)
t1_mask_cleaned = ants.get_mask(t1_brain,cleanup=3)
t1_brain_cleaned = ants.mask_image(t1_n3,t1_mask_cleaned)
ants.plot_ortho_stack([t1_n3,t1_brain,t1_brain_cleaned])

## Tissue segmentation
Next we are going to run a step of identifying tissue types within the brain. The brain is typically divided up into three primary tissue types: 
1. grey matter (GM),
2. white matter (WM), and
3. cerebrospinal fluid (CSF)
Some of these tissue types can be further broken down (for example cortical GM versus subcortical GM)

In [ ]:
tissue_seg = ants.atropos(a=t1_n3,x=t1_mask_cleaned,m="[0.3,1x1x1]", i='Kmeans[3]')

In [ ]:
ants.plot_ortho_stack([t1_n3,tissue_seg['segmentation'],tissue_seg['probabilityimages'][0],
tissue_seg['probabilityimages'][1],tissue_seg['probabilityimages'][2]])

## Brain parcellation
Another bit of labelling that is quite helpful is to identify brain regions for each voxel. These are really helpful when you wnat to define a region of interest (ROI) to obtain summary metrics for. This is primarily useful for calculating volumes of each structure, but also for multimodal analyiss when you want a high-resolution anatomical region defined to obtain summary metrics in low-resolution images where it is not possible to identify these regions directly.

There are many ways to do this. Many of the most widely-used approaches take a considerable amount of time (anywhere from an hour to a whole day). For your workbook, consider running another model (FreeSurfer, FastSurfer, ANTS, etc) on your own.
For the purpose of tdoay's demo, we are going to take a similar approach to the one used in the [brain extraction](#brain-extraction) section. This involves finding one labelled template atlas and registering it to our individual image.  

Many people use the [Automated Anatomical Labelling (AAL)](https://www.gin.cnrs.fr/en/tools/aal/) atlas for this purpose. We will first open that data up and have a look at it.

In [ ]:
# Determine template, rigid + SYN and then transfer labels over. 
aal_template_file = os.path.join(atlasdir,"AAL3v1_1mm.nii.gz")
aal_template = ants.image_read(aal_template_file)
aal_img_file = os.path.join(atlasdir,"single_subj_T1.nii")
aal_img = ants.image_read(aal_img_file)
ants.plot_ortho(aal_img,overlay=aal_template)


This time we are not only going to do an affine registration but also a nonrigid registration to warp the tempalte to match our image. This is quite a challenge for the algorithm, as the AAL tempalte is based on a single healthy 27 year old, while our subjects come from an Alzheimer's Disease cohort where most of the population are over 70.

This will take about four or five minutes to run. You are welcome to chop and change the setting to see if you can get improved registration..

In [ ]:
t1_mask_dil = ants.iMath(t1_mask_cleaned,'MD',2)
aal_affine_init_file = ants.affine_initializer(fixed_image = t1_n3,moving_image = aal_img)
syn_aal = ants.registration(fixed=t1_n3,moving=aal_img,mask=t1_mask_dil,
type_of_transform="SyN",initial_transform=aal_affine_init_file,
reg_iterations=(60,30,15))

In [ ]:
ants.plot_ortho_stack([t1_n3,syn_aal['warpedmovout']])

For the most part, the iamges are aligned. Do you notice the wavy bits in the bright fat regions near the skull in the warped tempalte? That is likely becuase we have told the algorithm to focus on the brain and not the surrounding areas so these regions do not have to fit very well to get a good alignment. 

The next step is to use the results from the registration to warp the labels into our image. We are only using one subject to do this, where specific varioations in anatomy between your source of the labels and the image you want to label. In fact, many groups now use an approach where they have tens to hundreds of labelled templates that they do this process to for the image, resulting in the same number of candidate labellings. Differences are resolved using a label fusion step to take the consensus information from the different candidate labellings and generate a final parecellation. This process is often called _multi atlas label fusion_.

In [ ]:
t1_aal_label = ants.apply_transforms(fixed = t1_n3, moving = aal_template,transformlist=syn_aal['fwdtransforms'],interpolator="genericLabel")
ants.plot_ortho(t1_n3,overlay=t1_aal_label)
bids_desc = "aal-resampled"
t1_aal_resampled_name = os.path.join(subject_root,"anat","sub-" + subject_id + "_desc-" + bids_desc + ".nii.gz")
ants.image_write(syn_aal['warpedmovout'],t1_aal_resampled_name)
bids_desc = "aal-labels"
t1_aal_label_name = os.path.join(subject_root,"anat","sub-" + subject_id + "_desc-" + bids_desc + ".nii.gz")
ants.image_write(t1_aal_label,t1_aal_label_name)

The below plot comes from using the freeSurfer pipeline to do the parcellation. This approach takes 3-6 hours to run on machines, and you can see the substanial imprvoement in the images as a result. 

In [ ]:
# See if we can clean this up, perhaps nibabel to read in?
# Also do the label image trick.
fsdir="/Users/davecash/Data/freesurfer"
t1_fs_nu_name  = os.path.join(fsdir,subject_id.replace("-","_"),"mri","nu.mgz")
t1_fs_nu = ants.image_read(t1_fs_nu_name)
t1_fs_aparc_name = os.path.join(fsdir,subject_id.replace("-","_"),"mri","aparc+aseg.mgz")
t1_fs_aparc = ants.image_read(t1_fs_aparc_name)
ants.plot_ortho(t1_fs_nu,overlay=t1_fs_aparc)

So we have parcellations. One valuable measure is the volume of each of the structures identified in the parcellation, just as we obtained from teh tissue segmentation above. 


## Co Registration
We have seen registration used above to match a standard atlas to our image. Now we are going to use it to align our PET data with our MRI data. 
Registration is used for many pipelines. The main goal is to determine the optimal mapping to align corresponding anatomy between it can be used to align different modalities together (for example PET to MR) to a standard space (such as MNI152) and to

In [ ]:
av45_name = os.path.join(subject_root,"pet")
av45 = ants.image_read(av45_name)